In [1]:
import json
import numpy as np
import os
import pickle
import shutil
import sqlite3
from collections import defaultdict
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret
import warnings
from pandas.core.common import SettingWithCopyWarning
from logzero import logger
from pandas.tseries.offsets import MonthEnd
from requests import get
import calendar
import regex as re
import pandas as pd

In [2]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)

In [3]:
query_zip = """
SELECT *

FROM program

LEFT JOIN zipcode on program.state_id == zipcode.state_id

WHERE is_entire_state == 1
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

state_df = pd.read_sql_query(query_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [4]:
query_city_zip = """
SELECT *

FROM program_city

LEFT JOIN program

on program_city.program_id == program.id

LEFT JOIN zipcode on program_city.city_id == zipcode.city_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

city_df = pd.read_sql_query(query_city_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [5]:
query_county_zip = """
SELECT *

FROM program_county

LEFT JOIN program

on program_county.program_id == program.id

LEFT JOIN zipcode on program_county.county_id == zipcode.county_id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

county_df = pd.read_sql_query(query_county_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [6]:
query_utility_zip = """
SELECT *

FROM program_utility

LEFT JOIN program

on program_utility.program_id == program.id

LEFT JOIN utility_zipcode

on program_utility.utility_id == utility_zipcode.utility_id

LEFT JOIN zipcode on utility_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

utility_df = pd.read_sql_query(query_utility_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [7]:
query_zip_zip = """
SELECT *

FROM program_zipcode

LEFT JOIN program

on program_zipcode.program_id == program.id

INNER JOIN zipcode on program_zipcode.zipcode_id == zipcode.id

WHERE is_entire_state == 0
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_df = pd.read_sql_query(query_zip_zip, conn_proj)
    
conn_proj.close

<function Connection.close>

In [8]:
county_df = county_df.loc[:,~county_df.columns.duplicated()]
city_df = city_df.loc[:,~city_df.columns.duplicated()]
state_df = state_df.loc[:,~state_df.columns.duplicated()]
zip_df = zip_df.loc[:,~zip_df.columns.duplicated()]
utility_df = utility_df.loc[:,~utility_df.columns.duplicated()]

keep_cols = list(
    set(zip_df.columns).intersection(
    set(utility_df.columns)).intersection(
    set(county_df.columns)).intersection(
    set(city_df.columns)).intersection(
    set(state_df.columns))
)



dsire_by_zip = pd.concat([zip_df[keep_cols],
                          utility_df[keep_cols],
                          county_df[keep_cols],
                          city_df[keep_cols],
                          state_df[keep_cols]])

In [9]:
query_tech = """
SELECT * 

FROM program_technology

WHERE program_technology.technology_id == 7
"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

tech_df = pd.read_sql_query(query_tech, conn_proj)

conn_proj.close

<function Connection.close>

In [10]:
check_list = tech_df.program_id.to_list()

In [11]:
dsire_PV = dsire_by_zip[dsire_by_zip.id.isin(check_list)]

In [12]:
dsire_PV['start_date'] = dsire_PV.start_date.fillna(dsire_PV.created_ts)
dsire_PV['end_date'] = dsire_PV.end_date.fillna(datetime.strptime(cfg["max_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
diff_df = dsire_PV.dropna(subset = ['start_date', 'zipcode']).reset_index()
diff_df['zipcode'] = diff_df.zipcode.apply(lambda n: str(int(n)).zfill(5))
diff_df.head()

,index,fundingsource,published,end_date_text,name,county_id,id,created_ts,implementing_sector_id,is_entire_state,...,latitude,fromSir,summary,program_type_id,longitude,start_date,updated_ts,administrator,created_by_user_id,program_category_id
0,0,None,0,None,Xcel Energy - Net Metering,2814.0,9,2003-09-05 00:00:00,3,0,...,40.0,0,Xcel Energy offers a net-metering program for ...,37,-105.0,2003-09-05 00:00:00,2004-11-22 00:00:00,None,534,2
1,1,None,0,None,Xcel Energy - Net Metering,2814.0,9,2003-09-05 00:00:00,3,0,...,40.0,0,Xcel Energy offers a net-metering program for ...,37,-105.0,2003-09-05 00:00:00,2004-11-22 00:00:00,None,534,2
2,2,None,0,None,Xcel Energy - Net Metering,2814.0,9,2003-09-05 00:00:00,3,0,...,40.0,0,Xcel Energy offers a net-metering program for ...,37,-105.0,2003-09-05 00:00:00,2004-11-22 00:00:00,None,534,2
3,3,None,0,None,Xcel Energy - Net Metering,2814.0,9,2003-09-05 00:00:00,3,0,...,40.0,0,Xcel Energy offers a net-metering program for ...,37,-105.0,2003-09-05 00:00:00,2004-11-22 00:00:00,None,534,2
4,4,None,0,None,Xcel Energy - Net Metering,2814.0,9,2003-09-05 00:00:00,3,0,...,40.0,0,Xcel Energy offers a net-metering program for ...,37,-105.0,2003-09-05 00:00:00,2004-11-22 00:00:00,None,534,2


In [13]:
query_sector = """
SELECT * 

FROM program_sector

"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

sector_df = pd.read_sql_query(query_sector, conn_proj)

conn_proj.close

<function Connection.close>

In [14]:
diff_df = diff_df.merge(sector_df, left_on = 'id', right_on = 'program_id', how = 'left')

In [15]:
diff_df = diff_df[diff_df.sector_id.isin([9,23])]
diff_df.sample(5)

,index,fundingsource,published,end_date_text,name,county_id,id,created_ts,implementing_sector_id,is_entire_state,...,summary,program_type_id,longitude,start_date,updated_ts,administrator,created_by_user_id,program_category_id,program_id,sector_id
1378228,287368,None,0,None,Sustainable Development Fund Solar PV Grant Pr...,175.0,662,2001-12-17 00:00:00,2,1,...,<B>Note: This program is temporarily closed to...,88,-80.0,2001-12-01 00:00:00,2007-01-22 00:00:00,Sustainable Development Fund,534,1,662.0,9.0
1838065,510324,None,1,None,Solar Energy Sales Tax Exemption,1711.0,1218,2005-07-14 00:00:00,1,1,...,"<p>&#10;&#9;In Minnesota, solar-energy systems...",81,-96.0,2005-07-14 00:00:00,2021-03-09 21:16:48,Department of Revenue,534,1,1218.0,9.0
1269271,243831,None,1,None,Property Tax Exclusion for Solar Energy Systems,3066.0,558,2001-04-19 00:00:00,1,1,...,<p><span>Section 73 of the California Revenue ...,78,-122.0,2001-04-19 00:00:00,2021-03-08 20:31:12,California State Board of Equalization,534,1,558.0,9.0
2974409,1161835,None,1,None,Solar Energy Systems Tax Credit (Personal),1527.0,5191,2012-06-05 00:00:00,1,1,...,<p><span>Iowa offers an individual tax credit ...,31,-93.0,2012-01-01 05:00:00,2020-08-12 16:53:54,Iowa Department of Revenue,534,1,5191.0,9.0
1219675,234027,None,1,None,Net Metering,2445.0,536,2001-05-16 00:00:00,1,1,...,"<p>In April 2001, Arkansas enacted legislation...",37,-92.0,2001-05-16 00:00:00,2018-11-30 15:36:11,None,534,2,536.0,9.0


In [16]:
#datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)")
diff_df.columns

Index(['index', 'fundingsource', 'published', 'end_date_text', 'name',
       'county_id', 'id', 'created_ts', 'implementing_sector_id',
       'is_entire_state', 'end_date', 'budget', 'city_id', 'websiteurl',
       'start_date_text', 'additional_technologies', 'zipcode', 'state_id',
       'code', 'latitude', 'fromSir', 'summary', 'program_type_id',
       'longitude', 'start_date', 'updated_ts', 'administrator',
       'created_by_user_id', 'program_category_id', 'program_id', 'sector_id'],
      dtype='object')

In [28]:
program_map_df = diff_df[['program_type_id','zipcode','start_date','end_date','name']]
program_map_df['s_d'] = pd.to_datetime(program_map_df.start_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))
program_map_df['e_d'] = pd.to_datetime(program_map_df.end_date).apply(lambda d: d - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))

program_map_df['start_month'] = pd.to_datetime(program_map_df.start_date).dt.strftime('%Y-%m')
program_map_df['start_month'] = pd.to_datetime(program_map_df['start_month']) + MonthEnd(1)



In [29]:
program_map_df['nb_months'] = program_map_df['start_month'].apply(lambda x: ((x - datetime.strptime(cfg["min_date"], "%d-%b-%Y (%H:%M:%S.%f)"))/np.timedelta64(1, 'M')))
program_map_df['nb_months'] = program_map_df['nb_months'].astype(int)

In [30]:
program_map_df["s_d"] = program_map_df.s_d.dt.days
program_map_df["e_d"] = program_map_df.e_d.dt.days
program_map_df.head()

,program_type_id,zipcode,start_date,end_date,name,s_d,e_d,start_month,nb_months
1,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
3,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
5,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
7,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
9,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67


In [31]:
def keys_exists(element, *keys):
    '''
    Check if *keys (nested) exists in `element` (dict).
    '''
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
        except KeyError:
            return False
    return True




program_map = defaultdict(dict)
    
for a, b, c, d in zip(program_map_df.program_type_id,program_map_df.zipcode,program_map_df.s_d,program_map_df.nb_months):
    
    if keys_exists(program_map, a, b):
        program_map[a][b].append([c,d])
    else:
        program_map[a][b] = [[c,d]]

In [32]:
with open('../data/program_zip_dates.p', 'wb') as fp:
    pickle.dump(program_map, fp, protocol=pickle.HIGHEST_PROTOCOL)

notebook was appearing in different folder. below code should correct issues with files not being found. 

In [33]:
# import os
# os.chdir("../../solar-697/ru")
# cwd = os.getcwd()  # Get the current working directory (cwd)
# files = os.listdir(cwd)  # Get all the files in that directory
# print("Files in %r: %s" % (cwd, files))

In [34]:
program_map_df['start_month'] = program_map_df['start_month'].dt.strftime('%Y-%m-%d')


In [45]:
conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

program_map_df.to_sql("programs_map_2", conn_proj, if_exists="replace", index=False)

conn_proj.commit

OperationalError: database is locked

In [39]:
program_map_df.head()

,program_type_id,zipcode,start_date,end_date,name,s_d,e_d,start_month,nb_months
1,37,80002,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
3,37,80003,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
5,37,80004,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
7,37,80005,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
9,37,80007,2003-09-05 00:00:00,2019-12-31 00:00:00,Xcel Energy - Net Metering,2044,8005,2003-09-30,67
